In [ ]:
import os.path
import time
import re

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# cuda_available = torch.cuda.is_available()
# dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

device = "cuda" if torch.cuda.is_available() else "cpu"
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
print(f"Using {device} device")

# torch.cuda.empty_cache()

# Tensor = torch.Tensor
# device = torch.device("cpu")

In [ ]:
import torchvision
import tqdm

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import torchvision.transforms as transforms

FFHQ_DIR = "/home/andresokol/data/compressed"
STRUCTURE_ROOT = f"/home/andresokol/data/orientation"
RENDERED_ROOT = f"/home/andresokol/rendered/v1"
MASK_ROOT = f"/home/andresokol/data/masks"

In [ ]:
class DownsampleBlock(nn.Module):
    def __init__(self):
        super().__init__()

        def _block(channels_in, channels_out):
            return [
                nn.Conv2d(channels_in, channels_out, kernel_size=3, padding=1, stride=2),
                nn.ReLU(),
            ]

        self.layers = nn.Sequential(
            *_block(3, 6),
            *_block(6, 12),
            *_block(12, 24),
            *_block(24, 48),
            *_block(48, 48),
        )
#         self.layers = nn.Sequential(
#             *_block(3, 3),
#             *_block(3, 3),
#             *_block(3, 3),
#             *_block(3, 3),
#             *_block(3, 3),
#         )

    def forward(self, x: Tensor) -> Tensor:
        return self.layers(x)

In [ ]:
# PyTorch version of ResNet
# https://github.com/pytorch/vision/blob/cddad9ca3822011548e18342f52a3e9f4724c2dd/torchvision/models/resnet.py#L88


class ResBlk(nn.Module):
    def __init__(self, channels) -> None:
        super().__init__()
        self.conv_1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.batch_norm_1 = nn.BatchNorm2d(channels)
        self.conv_2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.batch_norm_2 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x: Tensor) -> Tensor:
        out = self.conv_1(x)
        out = self.batch_norm_1(out)
        out = self.relu(out)

        out = self.conv_2(out)
        out = self.batch_norm_2(out)

        out += x
        out = self.relu(out)

        return out

In [ ]:
# E_f = nn.Sequential(
#     DownsampleBlock(),
#     *(ResBlk() for _ in range(4)),
# ).to(device)

# E_r = nn.Sequential(
#     DownsampleBlock(),
#     *(ResBlk() for _ in range(4)),
# ).to(device)

# E_ = nn.Sequential(
#     ResBlk(),
#     ResBlk(),
# ).to(device)

In [ ]:
class GeneratorFake(nn.Module):
    def __init__(self):
        super().__init__()

        def _upscale_conv(channels_in, channels_out):
            return (
                nn.BatchNorm2d(channels_in),
                nn.ConvTranspose2d(channels_in, channels_out, kernel_size=3, stride=2, padding=1, output_padding=1),  # is this correct?
                nn.LeakyReLU(),
            )

        self.resblk1 = ResBlk(48)
        self.resblk2 = ResBlk(48)
        self.upscale_layers = nn.Sequential(
            *_upscale_conv(48, 48),
            *_upscale_conv(48, 24),
            *_upscale_conv(24, 12),
            *_upscale_conv(12, 6),
            *_upscale_conv(6, 3),
        )
#         self.upscale_layers = nn.Sequential(
#             *_upscale_conv(3, 3),
#             *_upscale_conv(3, 3),
#             *_upscale_conv(3, 3),
#             *_upscale_conv(3, 3),
#             *_upscale_conv(3, 3),
#         )

        self.out_conv = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x: Tensor) -> Tensor:
        out = self.resblk1(x)
        out = self.resblk2(out)
        out = self.upscale_layers(out)
        # print(out.shape)

        # for i in range(5):
        #     out = self.upscale_layers[i](out)
        # print(out.shape)

        return torch.tanh(self.out_conv(out))


# G_f = GeneratorFake().to(device)
# G_f = nn.Sequential(
#     *(ResBlk() for _ in range(2)),
#     *(nn.ConvTranspose2d(3, 3, kernel_size=3, stride=2, padding=1) for _ in range(5)),
#     # nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
# )

In [ ]:
# from https://github.com/NVlabs/SPADE

# Creates SPADE normalization layer based on the given configuration
# SPADE consists of two steps. First, it normalizes the activations using
# your favorite normalization method, such as Batch Norm or Instance Norm.
# Second, it applies scale and bias to the normalized output, conditioned on
# the segmentation map.
# The format of |config_text| is spade(norm)(ks), where
# (norm) specifies the type of parameter-free normalization.
#       (e.g. syncbatch, batch, instance)
# (ks) specifies the size of kernel in the SPADE module (e.g. 3x3)
# Example |config_text| will be spadesyncbatch3x3, or spadeinstance5x5.
# Also, the other arguments are
# |norm_nc|: the #channels of the normalized activations, hence the output dim of SPADE
# |label_nc|: the #channels of the input semantic map, hence the input dim of SPADE
class SPADE(nn.Module):
    def __init__(self, config_text, norm_nc, label_nc):
        super().__init__()

        assert config_text.startswith('spade')
        parsed = re.search('spade(\D+)(\d)x\d', config_text)
        param_free_norm_type = str(parsed.group(1))
        ks = int(parsed.group(2))

        if param_free_norm_type == 'instance':
            self.param_free_norm = nn.InstanceNorm2d(norm_nc, affine=False)
        # elif param_free_norm_type == 'syncbatch':
        #     self.param_free_norm = SynchronizedBatchNorm2d(norm_nc, affine=False)
        elif param_free_norm_type == 'batch':
            self.param_free_norm = nn.BatchNorm2d(norm_nc, affine=False)
        else:
            raise ValueError('%s is not a recognized param-free norm type in SPADE'
                             % param_free_norm_type)

        # The dimension of the intermediate embedding space. Yes, hardcoded.
        nhidden = 128

        pw = ks // 2
        self.mlp_shared = nn.Sequential(
            nn.Conv2d(label_nc, nhidden, kernel_size=ks, padding=pw),
            nn.ReLU()
        )
        self.mlp_gamma = nn.Conv2d(nhidden, norm_nc, kernel_size=ks, padding=pw)
        self.mlp_beta = nn.Conv2d(nhidden, norm_nc, kernel_size=ks, padding=pw)

    def forward(self, x, segmap):

        # Part 1. generate parameter-free normalized activations
        normalized = self.param_free_norm(x)

        # Part 2. produce scaling and bias conditioned on semantic map
        segmap = F.interpolate(segmap, size=x.size()[2:], mode='nearest')
        actv = self.mlp_shared(segmap)
        gamma = self.mlp_gamma(actv)
        beta = self.mlp_beta(actv)

        # apply scale and bias
        out = normalized * (1 + gamma) + beta

        return out


# ResNet block that uses SPADE.
# It differs from the ResNet block of pix2pixHD in that
# it takes in the segmentation map as input, learns the skip connection if necessary,
# and applies normalization first and then convolution.
# This architecture seemed like a standard architecture for unconditional or
# class-conditional GAN architecture using residual block.
# The code was inspired from https://github.com/LMescheder/GAN_stability.
class SPADEResnetBlock(nn.Module):
    def __init__(self, fin, fout, segmap_channels):
        super().__init__()
        # Attributes
        self.learned_shortcut = (fin != fout)
        fmiddle = min(fin, fout)
        
        # create conv layers
        self.conv_0 = nn.Conv2d(fin, fmiddle, kernel_size=3, padding=1)
        self.conv_1 = nn.Conv2d(fmiddle, fout, kernel_size=3, padding=1)
        if self.learned_shortcut:
            self.conv_s = nn.Conv2d(fin, fout, kernel_size=1, bias=False)

        # apply spectral norm if specified
        # if 'spectral' in opt.norm_G:
        #     self.conv_0 = spectral_norm(self.conv_0)
        #     self.conv_1 = spectral_norm(self.conv_1)
        #     if self.learned_shortcut:
        #         self.conv_s = spectral_norm(self.conv_s)

        # define normalization layers
#         spade_config_str = opt.norm_G.replace('spectral', '')
        spade_config_str = "spadebatch3x3"
        self.norm_0 = SPADE(spade_config_str, fin, segmap_channels)
        self.norm_1 = SPADE(spade_config_str, fmiddle, segmap_channels)
        if self.learned_shortcut:
            self.norm_s = SPADE(spade_config_str, fin, segmap_channels)

    # note the resnet block with SPADE also takes in |seg|,
    # the semantic segmentation map as input
    def forward(self, x, seg):
        x_s = self.shortcut(x, seg)

        dx = self.conv_0(self.actvn(self.norm_0(x, seg)))
        dx = self.conv_1(self.actvn(self.norm_1(dx, seg)))

        out = x_s + dx

        return out

    def shortcut(self, x, seg):
        if self.learned_shortcut:
            x_s = self.conv_s(self.norm_s(x, seg))
        else:
            x_s = x
        return x_s

    def actvn(self, x):
        return F.leaky_relu(x, 2e-1)

In [ ]:
class GeneratorReal(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.resblk1 = ResBlk(48)
        self.resblk2 = ResBlk(48)

        mat_features_channels = 48
        
        self.spade1 = SPADEResnetBlock(48, 48, mat_features_channels)
        self.spade2 = SPADEResnetBlock(48, 24, mat_features_channels)
        self.spade3 = SPADEResnetBlock(24, 12, mat_features_channels)
        self.spade4 = SPADEResnetBlock(12, 6, mat_features_channels)
        self.spade5 = SPADEResnetBlock(6, 3, mat_features_channels)
        self.upsample = nn.Upsample(scale_factor=2)

        self.out_conv = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x: Tensor, seg: Tensor) -> Tensor:
        out = self.resblk1(x)
        out = self.resblk2(out)

        out = self.upsample(out)
        out = self.spade1(out, seg)

        out = self.upsample(out)
        out = self.spade2(out, seg)
        
        out = self.upsample(out)
        out = self.spade3(out, seg)
        
        out = self.upsample(out)
        out = self.spade4(out, seg)
        
        out = self.upsample(out)
        out = self.spade5(out, seg)
        
        return torch.tanh(self.out_conv(out))

# G_r = GeneratorReal().to(device)

In [ ]:
class EncoderMaterial(nn.Module):
    def __init__(self):
        super().__init__()
        def _block(channels_in, channels_out):
            return [
                nn.Conv2d(channels_in, channels_out, kernel_size=3, padding=1, stride=2),
                nn.ReLU(),
            ]

        
        self.layers = nn.Sequential(
            *_block(3, 6),
            *_block(6, 12),
            *_block(12, 24),
            *_block(24, 48),
            *_block(48, 48), # [..., 48, 8, 8]
            nn.AvgPool2d(kernel_size=8), # [..., 48, 1, 1]
        )
    
    def forward(self, x: Tensor, mask: Tensor):
        out = self.layers(x)
#         print(mask.shape, out.shape)
        return torch.where(mask, out, torch.zeros_like(out))

# E_m = EncoderMaterial().to(device)

In [ ]:
# x = torch.rand((8, 3, 256, 256))
# mask = torch.rand((8, 1, 256, 256)) > 0.5
# emb = torch.rand((8, 48, 8, 8))

# m = EncoderMaterial()(x, mask)
# m.shape

In [ ]:
# GeneratorReal()(emb, m).shape

In [ ]:
# VGG architecter, used for the perceptual loss using a pretrained VGG network
class VGG19(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super().__init__()
        vgg_pretrained_features = torchvision.models.vgg19(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        for x in range(2):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(2, 7):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(7, 12):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(12, 21):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        for x in range(21, 30):
            self.slice5.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h_relu1 = self.slice1(X)
        h_relu2 = self.slice2(h_relu1)
        h_relu3 = self.slice3(h_relu2)
        h_relu4 = self.slice4(h_relu3)
        h_relu5 = self.slice5(h_relu4)
        out = [h_relu1, h_relu2, h_relu3, h_relu4, h_relu5]
        return out

# Perceptual loss that uses a pretrained VGG network
class VGGLoss(nn.Module):
    transforms = transforms.Compose([
        transforms.Normalize(mean=[0., 0., 0.], std=[1/0.5, 1/0.5, 1/0.5]),
        transforms.Normalize(mean=[-0.5, -0.5, -0.5], std=[1, 1, 1]),
        transforms.Resize((224, 224)), # better without centercrop?
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    def __init__(self):
        super(VGGLoss, self).__init__()
        self.vgg = VGG19().to(device)
        self.criterion = nn.L1Loss()
        self.weights = [1.0 / 32, 1.0 / 16, 1.0 / 8, 1.0 / 4, 1.0]

    def forward(self, x, y):
        x_resized = self.transforms(x)
        y_resized = self.transforms(y)
        
        x_vgg, y_vgg = self.vgg(x_resized), self.vgg(y_resized)
        loss = 0
        for i in range(len(x_vgg)):
            loss += self.weights[i] * self.criterion(x_vgg[i], y_vgg[i].detach())
        return loss

In [ ]:
# https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/stargan/models.py

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.layers = nn.Sequential(
            *discriminator_block(3, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, kernel_size=4, padding=1, bias=False),
        )
        self.layers2 = nn.Sequential(
            nn.Flatten(),
            #nn.Linear(1024, 1), # 512x512
            nn.Linear(256, 1),  # 256x256
        )

    def forward(self, img: Tensor) -> Tensor:
        # combined = torch.cat((img_a, img_b), dim=1)
        # out = img
        # # print("Discriminator start", out.shape)
        #
        # for layer in self.layers:
        #     out = layer(out)
        #     # print(layer, out.shape)

        out = self.layers(img)
        out = self.layers2(out)
        
        return out

In [ ]:
class EmbeddingDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.layers = nn.Sequential(
            *discriminator_block(48, 96, normalization=False),
            *discriminator_block(96, 192),
            nn.Flatten(),
            nn.Linear(192 * 2 * 2, 1),
        )
    
    def forward(self, x: Tensor) -> Tensor:
        return self.layers(x)

In [ ]:
class Options:
    img_height = 256
    img_width = 256
    learning_rate = 0.0001  # from paper
    starting_epoch = 0
    total_epochs = 300
    batch_size = 4
    cpu_count = 2


opt = Options()

In [ ]:
class ImageDataset(Dataset):
    transform = transforms.Compose([
        transforms.Resize((opt.img_height, opt.img_width), transforms.InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    def __init__(self, mode="train"):
        self.files = []

        for i in range(70):
            for j in range(1000):
                img_dir = f"{i:02}000"
                img_name = f"{i:02}{j:03}"
                if os.path.exists(f"{FFHQ_DIR}/{img_dir}/{img_name}.jpg"):
                    if os.path.exists(f"{RENDERED_ROOT}/{img_dir}_{img_name}_base.png"):
                        if os.path.exists(f"{RENDERED_ROOT}/{img_dir}_{img_name}_structure.png"):
                            self.files.append((img_dir, img_name))

        print(f"Read {len(self.files)} images")
        # self.files = sorted(glob.glob(os.path.join(root, mode) + "/*.*"))
        # if mode == "train":
        #     self.files.extend(sorted(glob.glob(os.path.join(root, "test") + "/*.*")))

    def __getitem__(self, index):
        img_dir, img_name = self.files[index % len(self.files)]
        
        photo = Image.open(f"{FFHQ_DIR}/{img_dir}/{img_name}.jpg")
        photo.putalpha(255)

        render_base = Image.open(f"{RENDERED_ROOT}/{img_dir}_{img_name}_base.png")
        render_struct = Image.open(f"{RENDERED_ROOT}/{img_dir}_{img_name}_structure.png")

        render_base = Image.alpha_composite(photo, render_base).convert("RGB")
        render_struct = Image.alpha_composite(photo, render_struct).convert("RGB")
        
        photo_structure = Image.open(f"{STRUCTURE_ROOT}/{img_dir}/{img_name}.png")
        
        mask_structure = Image.open(f"{MASK_ROOT}/{img_dir}/{img_name}.jpg")
        mask_structure = mask_structure.resize((opt.img_height, opt.img_width))
        mask_structure = (np.array(mask_structure) > 128)[:,:,:1] # only last channel

        return {
            "photo_base": self.transform(photo.convert("RGB")),
            "photo_struct": self.transform(photo_structure),
            "mask": torch.from_numpy(mask_structure).permute((2, 0, 1)),
            "render_base": self.transform(render_base),
            "render_struct": self.transform(render_struct),
        }

    def __len__(self):
        return len(self.files)

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    ImageDataset(),
    batch_size=opt.batch_size,
    num_workers=opt.cpu_count,
    shuffle=True,
)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    print(classname)
    if hasattr(m, 'weight') and m.weight is not None and classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
    elif hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)

In [ ]:
def gan_loss(generated_scores: torch.Tensor,
             real_scores: torch.Tensor,
             for_generator: bool) -> torch.Tensor:
    if for_generator:
        return F.logsigmoid(-generated_scores).mean()
    
    return -F.logsigmoid(real_scores).mean() - F.logsigmoid(-generated_scores).mean()

In [ ]:
def discriminator_accuracy(generated_scores: torch.Tensor,
                           real_scores: torch.Tensor) -> float:
    return torch.cat((generated_scores < 0, real_scores > 0)).float().mean()

In [ ]:
def to_imgs(x: Tensor):
    # normalize: x -> (x - mu) / sigma
    # denormalize: x -> x * sigma + mu
    img = x.cpu().detach().permute((0, 2, 3, 1)).numpy()
    img = (img + 1) * 128
    return img.astype("uint8")

In [ ]:
def image_grid_to_writer(writer, x, label, iteration):
    grid = torchvision.utils.make_grid(x, normalize=True, nrow=opt.batch_size)
    writer.add_image(label, grid, iteration, )

In [ ]:
class MySuperNetwork(nn.Module):
    lambda_vgg = 10
    
    def __init__(self):
        super().__init__()
        self.E_f = nn.Sequential(
            DownsampleBlock(),
            *(ResBlk(48) for _ in range(4)),
        )
        self.E_shared = nn.Sequential(
            ResBlk(48),
            ResBlk(48),
        )
        self.G_f = GeneratorFake()
        self.D_f = Discriminator()
        
        self.E_r = nn.Sequential(
            DownsampleBlock(),
            *(ResBlk(48) for _ in range(4)),
        )
        self.E_m = EncoderMaterial()
        self.G_r = GeneratorReal()
        self.D_r = Discriminator()
        
        self.D_embedding = EmbeddingDiscriminator()
        
        self.vgg_loss = VGGLoss()

    def create_optimizers(self, opt):
        gan_params = list(self.E_f.parameters())
        gan_params += list(self.E_shared.parameters())
        gan_params += list(self.G_f.parameters())
        
        gan_params += list(self.E_r.parameters())
        gan_params += list(self.E_m.parameters())
        gan_params += list(self.G_r.parameters())

        disc_params = list(self.D_f.parameters())
        disc_params += list(self.D_r.parameters())
        disc_params += list(self.D_embedding.parameters())

        gan_optimizer = torch.optim.Adam(gan_params, lr=opt.learning_rate)
        disc_optimizer = torch.optim.Adam(disc_params, lr=opt.learning_rate)

        return gan_optimizer, disc_optimizer

    def init_weights(self):
        self.E_f.apply(weights_init_normal)
        self.E_shared.apply(weights_init_normal)
        self.G_f.apply(weights_init_normal)
        self.D_f.apply(weights_init_normal)
        
        self.E_r.apply(weights_init_normal)
        self.E_m.apply(weights_init_normal)
        self.G_r.apply(weights_init_normal)
        self.D_r.apply(weights_init_normal)
    
    def _fake_branch(self, input_images):
        embedding = self.E_shared(self.E_f(input_images))
        generated = self.G_f(embedding)
        return generated
    
    def _real_branch(self, input_images, reference_images, masks):
        embedding = self.E_shared(self.E_r(input_images))
        material_embedding = self.E_m(reference_images, masks)
        generated = self.G_r(embedding, material_embedding)
        return generated
    
    def fake_branch_generator(self, input_images, ground_truth):
        generated = self._fake_branch(input_images)
        generated_scores = self.D_f(generated)

        losses = dict()
        losses["branch_F_generator_loss"] = gan_loss(generated_scores, None, for_generator=True)
        losses["VGG_F_loss"] = self.vgg_loss(generated, ground_truth) * self.lambda_vgg

        return generated, losses

    def fake_branch_discriminator(self, input_images, ground_truth):
        with torch.no_grad():
            generated = self._fake_branch(input_images)
            generated = generated.detach()
            generated.requires_grad_() # eh?

        generated_scores = self.D_f(generated)
        truth_scores = self.D_f(ground_truth)

        losses = {
            "D_F_loss": gan_loss(generated_scores, truth_scores, for_generator=False),
            "D_F_accuracy": discriminator_accuracy(generated_scores.detach(), truth_scores.detach()), # eh? detach?
        }
        return generated, losses

    def real_branch_generator(self, input_images, ground_truth, masks):
        generated = self._real_branch(input_images, ground_truth, masks)
        generated_scores = self.D_r(generated)
        
        losses = dict()
        losses["branch_R_generator_loss"] = gan_loss(generated_scores, None, for_generator=True)
        losses["VGG_R_loss"] = self.vgg_loss(generated, ground_truth) * self.lambda_vgg

        return generated, losses
    
    def real_branch_discriminator(self, input_images, ground_truth, masks):
        with torch.no_grad():
            generated = self._real_branch(input_images, ground_truth, masks)
            generated = generated.detach()
            generated.requires_grad_() # eh?

        generated_scores = self.D_r(generated)
        truth_scores = self.D_r(ground_truth)

        losses = {
            "D_R_loss": gan_loss(generated_scores, truth_scores, for_generator=False),
            "D_R_accuracy": discriminator_accuracy(generated_scores.detach(), truth_scores.detach()), # eh? detach?
        }
        return generated, losses
    
    def _both_branches(self, fake_input, fake_truths, real_input, real_truth, real_masks):
        embedding_fake = self.E_shared(self.E_f(fake_input))
        generated_fake = self.G_f(embedding_fake)
        
        embedding_real = self.E_shared(self.E_r(real_input))
        segmentation_mask = self.E_m(real_truth, real_masks)
        generated_real = self.G_r(embedding_real, segmentation_mask)
        
        return generated_fake, embedding_fake, generated_real, embedding_real
    
    def both_generators(self, fake_input, fake_truths, real_input, real_truth, real_masks):
        generated_fake, embedding_fake, generated_real, embedding_real = self._both_branches(
            fake_input, fake_truths, real_input, real_truth, real_masks)
        
        generated_fake_scores = self.D_f(generated_fake)
        truth_fake_scores = self.D_f(fake_truths)
        
        generated_real_scores = self.D_r(generated_real)
        truth_real_scores = self.D_r(real_truth)

        embedding_fake_scores = self.D_embedding(embedding_fake)
        embedding_real_scores = self.D_embedding(embedding_real)

        losses = {
            "generator_F_loss": gan_loss(generated_fake_scores, None, for_generator=True),
            "VGG_F_loss": self.vgg_loss(generated_fake, fake_truths) * self.lambda_vgg,
            
            "generator_R_loss": gan_loss(generated_real_scores, None, for_generator=True),
            "VGG_R_loss": self.vgg_loss(generated_real, real_truth) * self.lambda_vgg,
            
            # distinguishing both branches therefore False to compute all scores
            "-D_EMB_loss": -gan_loss(embedding_fake_scores, embedding_real_scores, for_generator=False),
        }
        accuracies = {
            "D_EMB_accuracy": discriminator_accuracy(embedding_fake_scores.detach(),
                                                     embedding_real_scores.detach()),
        }
        return generated_fake, generated_real, losses, accuracies
    
    def both_discriminators(self, fake_input, fake_truths, real_input, real_truth, real_masks):
        with torch.no_grad():
            outputs = self._both_branches(
                fake_input,
                fake_truths,
                real_input,
                real_truth,
                real_masks
            )
            generated_fake = outputs[0].detach()
            embedding_fake = outputs[1].detach()
            generated_real = outputs[2].detach()
            embedding_real = outputs[3].detach()

        generated_fake_scores = self.D_f(generated_fake)
        truth_fake_scores = self.D_f(fake_truths)
        
        generated_real_scores = self.D_r(generated_real)
        truth_real_scores = self.D_r(real_truth)

        embedding_fake_scores = self.D_embedding(embedding_fake)
        embedding_real_scores = self.D_embedding(embedding_real)

        losses = {
            "D_F_loss": gan_loss(generated_fake_scores, truth_fake_scores, for_generator=False),
            "D_R_loss": gan_loss(generated_real_scores, truth_real_scores, for_generator=False),
            "D_EMB_loss": gan_loss(embedding_fake_scores, embedding_real_scores, for_generator=False),
        }
        accuracies = {
            "D_F_accuracy": discriminator_accuracy(generated_fake_scores.detach(),
                                                   truth_fake_scores.detach()),
            "D_R_accuracy": discriminator_accuracy(generated_real_scores.detach(),
                                                   truth_real_scores.detach()),
            "D_EMB_accuracy": discriminator_accuracy(embedding_fake_scores.detach(),
                                                     embedding_real_scores.detach()),
        }

        return generated_fake, generated_real, losses, accuracies

            
        
network = MySuperNetwork().to(device)

In [ ]:
if opt.starting_epoch == 0:
    network.init_weights()

In [ ]:
gan_optimizer, disc_optimizer = network.create_optimizers(opt)
log_iter = 0

In [ ]:
import fractions

discriminator_freq = fractions.Fraction(1)

max_discriminator_freq = fractions.Fraction(32)
min_discriminator_freq = fractions.Fraction(1, 16)

def should_run_generator(i: int, discriminator_freq: fractions.Fraction):
    if discriminator_freq > 1:
        return i % discriminator_freq.numerator == 0

    return i % (discriminator_freq.denominator * 2) != 0

In [ ]:
# https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
# https://github.com/hse-ds/iad-applied-ds/blob/master/2022/seminars/sem03/sem03-gan-task.ipynb
# https://colab.research.google.com/github/yandexdataschool/mlhep2019/blob/master/notebooks/day-6/06_GAN_faces_solution.ipynb#scrollTo=-Z7YAPGxp2Xl

losses_f = {}
losses_d_f = {}
losses_r = {}
losses_d_r = {}

min_accuracy = 0
losses, accuracies = {}, {}

for epoch in range(opt.starting_epoch, opt.total_epochs):
    with tqdm.auto.tqdm(total=len(dataloader), unit="batch", desc=f"Epoch {epoch}") as pbar:
        for i, batch in enumerate(dataloader):
            render_base = batch["render_base"].to(device)
            render_struct = batch["render_struct"].to(device)
            photo_base = batch["photo_base"].to(device)
            photo_struct = batch["photo_struct"].to(device)
            masks = batch["mask"].to(device)
            
#             if i % discriminator_freq != 0:
#                 gan_optimizer.zero_grad()
#                 generated_f, losses_f = network.fake_branch_generator(render_struct, render_base)
#                 g_f_loss = sum(losses_f.values())
#                 g_f_loss.backward()
#                 gan_optimizer.step()
#             else:
#                 disc_optimizer.zero_grad()
#                 generated_f, losses_d_f = network.fake_branch_discriminator(render_struct, render_base)
#                 losses_d_f["D_F_loss"].backward()
#                 disc_optimizer.step()
                
#                 if losses_d_f["D_F_accuracy"].item() > 0.9 and discriminator_freq < 32:
#                     discriminator_freq *= 2
#                 elif losses_d_f["D_F_accuracy"].item() < 0.8 and discriminator_freq > 2:
#                     discriminator_freq /= 2

#             if i % discriminator_freq != 0:
#                 gan_optimizer.zero_grad()
#                 generated_r, losses_r = network.real_branch_generator(photo_struct, photo_base, masks)
#                 g_r_loss = sum(losses_r.values())
#                 g_r_loss.backward()
#                 gan_optimizer.step()
#             else:
#                 disc_optimizer.zero_grad()
#                 generated_r, losses_d_r = network.real_branch_discriminator(photo_struct, photo_base, masks)
#                 losses_d_r["D_R_loss"].backward()
#                 disc_optimizer.step()
                
#                 if losses_d_r["D_R_accuracy"].item() > 0.9 and discriminator_freq < 32:
#                     discriminator_freq *= 2
#                 elif losses_d_r["D_R_accuracy"].item() < 0.8 and discriminator_freq > 2:
#                     discriminator_freq /= 2

            if should_run_generator(i, discriminator_freq):
                gan_optimizer.zero_grad()

                generated_f, generated_r, losses, accuracies = network.both_generators(
                    render_struct, render_base, photo_struct, photo_base, masks)

                generator_loss = sum(losses.values())
                generator_loss.backward()

                gan_optimizer.step()

                losses["total_generator_loss"] = generator_loss
            else:
                disc_optimizer.zero_grad()

                generated_f, generated_r, losses, accuracies = network.both_discriminators(
                    render_struct, render_base, photo_struct, photo_base, masks)

                discriminator_loss = sum(losses.values())
                discriminator_loss.backward()

                disc_optimizer.step()

                min_accuracy = min(accuracies.values()).item()
                if min_accuracy < 0.8 and discriminator_freq < max_discriminator_freq:
                    discriminator_freq *= 2
                elif min_accuracy > 0.9 and discriminator_freq > min_discriminator_freq:
                    discriminator_freq /= 2

            for d in [losses, accuracies]:
                for key, value in d.items():
                    writer.add_scalar(key, value.item(), log_iter)
                    
            if i % 20 == 0:
#                 image_grid_to_writer(writer, generated_f, "generated fake", log_iter)
#                 image_grid_to_writer(writer, render_base, "true fake", log_iter)
#                 image_grid_to_writer(writer, generated_r, "generated real", log_iter)
#                 image_grid_to_writer(writer, photo_base, "true real", log_iter)
                image_grid_to_writer(
                    writer,
                    torch.cat([generated_f, render_base, generated_r, photo_base]),
                    "images",
                    log_iter,
                )

            pbar.set_postfix(discriminator_freq=str(discriminator_freq), min_accuracy=min_accuracy)
            pbar.update(1)
            log_iter += 1

In [ ]:
x = np.random.randn(8, 8, 3)
(x>0)[:,:,:1].shape